In [452]:
import pymysql

db_name = "crescimento_covid"
db_host = "localhost"
db_username = "root"
db_password = "root"

try:
    conn = pymysql.connect(host = db_host,
                           port = int(3306),
                           user = db_username,
                           password = db_password,
                           db = db_name)
except e:
    print (e)
if conn:
    print ("connection successful")
else:
    print ("error")

connection successful


In [ ]:
import pandas as pd

regioes = {
    'Norte': ('AM', 'PA', 'AC', 'RR', 'RO', 'AP', 'TO'),
    'Nordeste': ('PE', 'MA', 'PI', 'CE', 'RN', 'PB', 'AL', 'SE', 'BA'),
    'Centro-Oeste': ('DF', 'GO', 'MT', 'MS'),
    'Sudeste': ('ES', 'MG', 'RJ', 'SP'),
    'Sul': ('PR', 'SC', 'RS')
}

def get_municipios_metropolitanos_PE():
    sql = f"SELECT DISTINCT(municipio) AS municipios FROM taxas_covid WHERE uf = 'PE' AND localizacao = 'Reg. Metropolitana'"
    result = pd.read_sql_query(sql, conn)
    return result['municipios']

def get_casos_acumulados_municipios_metropolitanos_PE(municipio):
    sql = f"SELECT casos_acumulados FROM taxas_covid WHERE semana = 14 AND municipio = '{str(municipio)}'"
    result = pd.read_sql_query(sql, conn)
    return result['casos_acumulados'][0]

map_municipio_metropolitano_PE_casos_acumulados = []
municipios_metropolitanos_PE = get_municipios_metropolitanos_PE()

for municipio in municipios_metropolitanos_PE:
    casos_acumulados_municipio_PE = get_casos_acumulados_municipios_metropolitanos_PE(municipio)

    if (casos_acumulados_municipio_PE >= 0):
        map_municipio_metropolitano_PE_casos_acumulados.append(
            {'municipio': municipio, 'casos_acumulados': casos_acumulados_municipio_PE}
        )

def get_casos_acumulados_por_regiao(estados):
    sql = f"SELECT SUM(casos_acumulados) AS casos_acumulados FROM taxas_covid WHERE semana = 14 AND uf IN {tuple(estados)}"
    result = pd.read_sql_query(sql, conn)
    return result['casos_acumulados'][0]

def get_obitos_acumulados_por_regiao(estados):
    sql = f"SELECT SUM(obitos_acumulados) AS obitos_acumulados FROM taxas_covid WHERE semana = 14 AND uf IN {tuple(estados)}"
    result = pd.read_sql_query(sql, conn)
    return result['obitos_acumulados'][0]

def get_obitos_novos_semana_por_regiao(estados):
    sql = f"SELECT SUM(obitos_novos_semana_epidemiologica) AS obitos_novos_semana FROM taxas_covid WHERE semana = 14 AND uf IN {tuple(estados)}"
    result = pd.read_sql_query(sql, conn)
    return result['obitos_novos_semana'][0]

In [455]:
from matplotlib.ticker import FuncFormatter

def formatar_milhoes(valor, pos):
    return f'{valor / 1e6:.1f}M'

def load_formatar_milhoes(ax):
    formatter = FuncFormatter(formatar_milhoes)
    ax.yaxis.set_major_formatter(formatter)


In [456]:
import tkinter
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

def abrir_graficos_linha():
    nova_janela = tkinter.Toplevel()
    nova_janela.state('zoomed')

    nova_janela.title("Dashboard Gráficos de Linha Covid 19")

    label = tkinter.Label(nova_janela, text="Esta é a nova janela!")
    label.pack()

In [ ]:
janela = tkinter.Tk()

def donothing():
   x = 0

menubar = tkinter.Menu(janela)
filemenu = tkinter.Menu(menubar, tearoff=0)
filemenu.add_command(label="Gráficos de Barra", command=donothing)
filemenu.add_command(label="Gráficos de Linha", command=abrir_graficos_linha)
filemenu.add_separator()
filemenu.add_command(label="Fechar", command=janela.quit)
menubar.add_cascade(label="Opções de Gráficos", menu=filemenu)

janela.config(menu=menubar)

janela.state('zoomed')

janela.title("Dashboard Covid 19")

altura_tela = janela.winfo_screenheight()
largura_tela = janela.winfo_screenwidth()

meu_dpi = 96

divisao_altura = (altura_tela - 80) / 2
divisao_largura = largura_tela / 2

figura = plt.figure(figsize=(divisao_largura / meu_dpi, divisao_altura / meu_dpi), dpi=meu_dpi)
ax = figura.add_subplot(111)

casos_acumulados_norte = get_casos_acumulados_por_regiao(regioes['Norte'])
casos_acumulados_nordeste = get_casos_acumulados_por_regiao(regioes['Nordeste'])
casos_acumulados_centro_oeste = get_casos_acumulados_por_regiao(regioes['Centro-Oeste'])
casos_acumulados_sudeste = get_casos_acumulados_por_regiao(regioes['Sudeste'])
casos_acumulados_sul = get_casos_acumulados_por_regiao(regioes['Sul'])

bar_labels_regioes = ['Norte', 'Nordeste', 'Centro-Oeste', 'Sudeste', 'Sul', 'Total']
bar_colors = ['tab:red', 'tab:blue', 'tab:purple', 'tab:orange', 'tab:brown', 'tab:green']
casos_acumulados_counts = [casos_acumulados_norte, casos_acumulados_nordeste, casos_acumulados_centro_oeste, casos_acumulados_sudeste, casos_acumulados_sul]
casos_acumulados_counts.append(sum(casos_acumulados_counts))

min_y = min(casos_acumulados_counts) / 1.2
max_y = max(casos_acumulados_counts) * 1.2

bar_container = ax.bar(bar_labels_regioes, casos_acumulados_counts, color=bar_colors)
ax.set(ylabel='Quantidades', title='Casos acumulados por região até 14° Semana de 2024', ylim=(min_y, max_y))
ax.bar_label(bar_container, fmt='{:,.0f}')
load_formatar_milhoes(ax)

canvas = FigureCanvasTkAgg(figura, janela)
canvas.get_tk_widget().grid(row=0, column=0)


# Segundo gráfico *****************************************************************************************************************************************

obitos_acumulados_norte = get_obitos_acumulados_por_regiao(regioes['Norte'])
obitos_acumulados_nordeste = get_obitos_acumulados_por_regiao(regioes['Nordeste'])
obitos_acumulados_centro_oeste = get_obitos_acumulados_por_regiao(regioes['Centro-Oeste'])
obitos_acumulados_sudeste = get_obitos_acumulados_por_regiao(regioes['Sudeste'])
obitos_acumulados_sul = get_obitos_acumulados_por_regiao(regioes['Sul'])

obitos_acumulados_counts = [obitos_acumulados_norte, obitos_acumulados_nordeste, obitos_acumulados_centro_oeste, obitos_acumulados_sudeste, obitos_acumulados_sul]
obitos_acumulados_counts.append(sum(obitos_acumulados_counts))

figura2 = plt.figure(figsize=(divisao_largura / meu_dpi, divisao_altura / meu_dpi), dpi=meu_dpi)
ax2 = figura2.add_subplot(111)

bar_container = ax2.bar(bar_labels_regioes, obitos_acumulados_counts, color=bar_colors)

min_y = min(obitos_acumulados_counts) / 1.2
max_y = max(obitos_acumulados_counts) * 1.2

ax2.set(ylabel='Quantidades', title='Obitos acumulados por região até 14° Semana de 2024', ylim=(min_y, max_y))
ax2.bar_label(bar_container, fmt='{:,.0f}')
ax2.yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))

canvas2 = FigureCanvasTkAgg(figura2, janela)
canvas2.get_tk_widget().grid(row=1, column=1)


# Terceiro gráfico *****************************************************************************************************************************************

percentual_obitos_norte = obitos_acumulados_norte / casos_acumulados_norte * 100
percentual_obitos_nordeste = obitos_acumulados_nordeste / casos_acumulados_nordeste * 100
percentual_obitos_centro_oeste = obitos_acumulados_centro_oeste / casos_acumulados_centro_oeste * 100
percentual_obitos_sudeste = obitos_acumulados_sudeste / casos_acumulados_sudeste * 100
percentual_obitos_sul = obitos_acumulados_sul / casos_acumulados_sul * 100

percentual_obitos = [percentual_obitos_norte, percentual_obitos_nordeste, percentual_obitos_centro_oeste, percentual_obitos_sudeste, percentual_obitos_sul]
percentual_soma_total = obitos_acumulados_counts[-1] / casos_acumulados_counts[-1] * 100
percentual_obitos.append(percentual_soma_total)

figura3 = plt.figure(figsize=(divisao_largura / meu_dpi, divisao_altura / meu_dpi), dpi=meu_dpi)
ax3 = figura3.add_subplot(111)

bar_container = ax3.bar(bar_labels_regioes, percentual_obitos, color=bar_colors)

min_y = min(percentual_obitos) / 1.2
max_y = max(percentual_obitos) * 1.2

ax3.set(ylabel='Percentuais', title='Análise de obitos em Relação aos Casos de COVID-19 \n por Região até a 14ª Semana de 2024', ylim=(min_y, max_y))
ax3.bar_label(bar_container, fmt='{:.2f}%')
ax3.yaxis.set_major_formatter(mtick.PercentFormatter())

canvas3 = FigureCanvasTkAgg(figura3, janela)
canvas3.get_tk_widget().grid(row=1, column=0)


# Quarto gráfico *****************************************************************************************************************************************

map_municipio_metropolitano_PE_casos_acumulados.sort(key=lambda x: x['casos_acumulados'], reverse=True)

map_municipio_metropolitano_PE_casos_acumulados = map_municipio_metropolitano_PE_casos_acumulados[:5]

meu_dpi -= 28

figura4 = plt.figure(figsize=(divisao_largura / meu_dpi, divisao_altura / meu_dpi), dpi=meu_dpi)
ax4 = figura4.add_subplot(111)

municipios = [item['municipio'] for item in map_municipio_metropolitano_PE_casos_acumulados]
casos_acumulados = [item['casos_acumulados'] for item in map_municipio_metropolitano_PE_casos_acumulados]

bar_labels_regioes = municipios
bar_colors = ['tab:red', 'tab:blue', 'tab:purple', 'tab:orange', 'tab:brown', 'tab:green']
bar_labels_regioes.append("Total")
casos_acumulados_counts = casos_acumulados
casos_acumulados_counts.append(sum(casos_acumulados_counts))

min_y = min(casos_acumulados_counts) / 1.2
max_y = max(casos_acumulados_counts) * 1.2

bar_container = ax4.bar(bar_labels_regioes, casos_acumulados_counts, color=bar_colors)
ax4.set(ylabel='Quantidades', title='Casos acumulados por top 5 regiõe metropolitanas \n de Pernambuco até 14° Semana de 2024', ylim=(min_y, max_y))
ax4.bar_label(bar_container, fmt='{:,.0f}')

plt.xticks(rotation=11)

canvas = FigureCanvasTkAgg(figura4, janela)
canvas.get_tk_widget().grid(row=0, column=1)

janela.mainloop()